In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

%matplotlib inline

## Download Dataset

In [ ]:
'''
!wget https://data-flair.s3.ap-south-1.amazonaws.com/Data-Science-Data/face-mask-dataset.zip
!unzip /content/face-mask-dataset.zip

!unzip /content/Dataset/train.zip
!unzip /content/Dataset/test.zip
'''

In [ ]:
'''
mask_train_dir = '/content/train/with_mask'
without_mask_train_dir = '/content/train/without_mask'

test_dir = '/content/test'

mask_train = os.listdir('/content/train/with_mask')
without_mask_train = os.listdir('/content/train/without_mask')
'''

In [ ]:
## Sample Image output
sample_image_mask = os.path.join(mask_train_dir, mask_train[1])
image_mask = cv2.imread(sample_image_mask)
image_mask = cv2.cvtColor(image_mask, cv2.COLOR_BGR2RGB)
plt.imshow(image_mask)

In [ ]:
def load_images(img_names, data_dir, model_size=None):
    """Loads images in a 4D array.

    Args:
        img_names: A list of images names.
        model_size: The input size of the model.
        data_dir: Directory to load data from

    Returns:
        A 4D NumPy array.
    """
    imgs = []

    for img_name in img_names:
        img = os.path.join(data_dir, img_name)
        img = cv2.imread(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, dsize=model_size, interpolation = cv2.INTER_AREA)
        img = np.array(img)
        #img = np.expand_dims(img, axis=0)
        imgs.append(img)

    imgs = np.array(imgs, dtype="float32")

    return imgs

In [ ]:
with_mask = load_images(mask_train, mask_train_dir, (224, 224))
without_mask = load_images(without_mask_train, without_mask_train_dir, (224, 224))

imgs = np.concatenate((with_mask, without_mask))
imgs.shape

In [ ]:
labels_mask = np.ones(shape=(with_mask.shape[0], 1), dtype = np.int32)
labels_without_mask = np.zeros(shape=(without_mask.shape[0], 1), dtype = np.int32)

labels = np.concatenate((labels_mask, labels_without_mask))

# Convert Labels to one-hot encoding
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)


In [ ]:
# Split train-data into train and validation sets
from sklearn.model_selection import train_test_split
train_x, validation_x, train_y, validation_y = train_test_split(imgs, labels, test_size=0.2, random_state=42)

## Train the model

In [3]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.utils import np_utils 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, Input
from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [4]:
def mask_detector(input_shape, num_classes):

    inputs = Input(input_shape)

    net = Conv2D(64, (3,3), padding = 'same')(inputs)
    net = BatchNormalization(axis = 3)(net)
    net = Activation('relu')(net)
    net = MaxPooling2D((2,2))(net)
  
    net = Conv2D(128, (3,3), padding = 'same')(inputs)
    net = BatchNormalization(axis = 3)(net)
    net = Activation('relu')(net)
    net = MaxPooling2D((2,2))(net)
  
    net = Flatten()(net)
    net = Dense(64, activation = 'relu')(net)
    net = Dropout(0.5)(net)

    net = Dense(num_classes, activation = 'softmax')(net)
  
    model = Model(inputs = inputs, outputs = net, name = 'Face_mask_detection')
    return model

In [5]:
input_shape = (224, 224, 3)

model = mask_detector(input_shape, 2)

model.summary()

Model: "FaceDtection"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 128)     3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 128)     512       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 128)     0         
_________________________________________________________________
flatten (Flatten)            (None, 1605632)           0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [ ]:
# Initialize the optimizer and compile the model
optimizer = Adam(lr=0.01, decay=5e-4)

model.compile(loss='binary_crossentropy', optimizer=optimizer,
             metrics=['accuracy'])

In [ ]:
earlystopper = EarlyStopping(patience=15, verbose=1)
checkpointer = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)
callback_list = [checkpointer, earlystopper]

data_augmentation = True
num_epochs = 10

if not data_augmentation:
    print('Not using data augmentation.')
    h = face_mask_model.fit(train_x, train_y,
                  batch_size = 64,
                  validation_data = (validation_x, validation_y),
                  callbacks = callback_list, 
                  epochs = num_epochs,
                  verbose = 1)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random zoom
        zoom_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_x)

    # Fit the model on the batches generated by datagen.flow().
    results = face_mask_model.fit_generator(datagen.flow(train_x, train_y,
                                     batch_size = 64),
                                     validation_data = (validation_x, validation_y),
                                     epochs = num_epochs, verbose = 1, workers = 4,
                                     callbacks = callback_list)


In [ ]:
# list all data in history
print(results.history.keys())

# summarize history for accuracy
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [6]:
## Load Model
model = load_model("model.h5")

W0728 19:33:42.202229 140606702257984 deprecation.py:323] From /home/vaibhav/my_project_dir/my_project_env/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [96]:
def face_detect(faces, frame):
    
    for i in range(len(faces)):
        x, y, w, h = faces[i]
        face_frame = frame[y:y+h,x:x+w]
        face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame = cv2.resize(face_frame, (224, 224))
        face_frame = np.array(face_frame)
        face_frame = np.expand_dims(face_frame, axis=0)
        face_frame =  preprocess_input(face_frame)
        
        preds = model.predict(face_frame)
        for pred in preds:
            (withoutMask, mask) = pred
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y- 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    
        cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)
    
    return frame


In [97]:
front_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#profile_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

video_capture = cv2.VideoCapture(0)

# Check if camera opened successfully
if (video_capture.isOpened()== False): 
    print("Error opening video stream or file")
else:
    print("Successful")
    
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = front_face_cascade.detectMultiScale(gray,
                                          scaleFactor=1.1,
                                          minNeighbors=5,
                                          minSize=(60, 60),
                                          flags=cv2.CASCADE_SCALE_IMAGE)

    frame = face_detect(faces, frame)
        
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()


Successful
